In [1]:
import numpy as np
import argparse
from copy import deepcopy
import gym
import torch
import time

In [14]:
import sys
sys.path.append(os.path.join(os.getcwd(), 'Simulator'))
sys.path.append(os.path.join(os.getcwd(), 'Wolptinger'))

In [15]:
from Wolptinger.ddpg import DDPG
from Wolptinger.utils import *
from Wolptinger.evaluator import Evaluator

from Simulator.WordCounting import WordCountingEnv

In [16]:
%load_ext autoreload
%autoreload 2

In [26]:
def train(num_iterations, agent, env, evaluate, validate_steps, output, max_episode_length=None, debug=False):
    agent.is_training = True
    step = episode = episode_steps = 0
    episode_reward = 0.
    observation = None

    while step < num_iterations:
        # reset if it is the start of episode
        if observation is None:
            # the original method use deepcopy here
            observation = env.reset()
            agent.reset(observation)
        
        # agent choose action
        if step < args.warmup:
            action = agent.random_action()
        else:
            action = agent.select_action(observation)
            print(action)
        
        
        next_state, reward, done, info = env.step(action)
        if max_episode_length and episode_steps >= max_episode_length - 1:
            done = True
        
        # agent observe and update policy
        agent.observe(reward, next_state, done)
        if step > args.warmup:
            agent.update_policy()
        
        # if debug:
        #     prCyan(f'{step}: reward is {reward}')
        
        # evaluate the model
        # I personally don't think this will work, also, this will let the model memory a wrong
        # action
        if evaluate is not None and validate_steps > 0 and step % validate_steps == 0:
            policy = lambda x: agent.select_action(x, decay_epsilon=False)
            validate_reward = evaluate(env, policy, debug=False, visualize=False, save=False)
            if debug: 
                prYellow('[Evaluate] Step_{:07d}: mean_reward:{}'.format(step, validate_reward))
            agent.s_t = observation
            agent.a_t = action

        # save intermidate training model
        if step % int(num_iterations/3) == 0:
            agent.save_model(output)
        
        # update models
        step += 1
        episode_steps += 1
        episode_reward += reward
        observation = next_state
        
        # end of episode
        if done:
            if debug:
                prLightPurple(f'#{episode}: episode_reward: {episode_reward} steps:{step}')
            
            # agent.memory.append(
            #     observation,
            #     agent.select_action(observation)
            #     0.,
            #     False
            # )

            # reset
            observation = None
            episode_steps = 0
            episode_reward = 0.
            episode += 1

In [18]:
def test(num_episodes, agent, env, evaluate, model_path, visualize=True, debug=False):

    agent.load_weights(model_path)
    agent.is_training = False
    agent.eval()
    policy = lambda x: agent.select_action(x, decay_epsilon=False)

    for i in range(num_episodes):
        validate_reward = evaluate(env, policy, debug=debug, visualize=visualize, save=False)
        if debug: 
            prYellow('[Evaluate] #{}: mean_reward:{}'.format(i, validate_reward))

In [21]:
class Arguments(object):
    def __init__(self):
        self.mode = 'train'
        # self.env = "InvertedPendulum-v2"
        self.env = 'WordCountingEnv'
        self.h1 = 128
        self.h2 = 128
        self.rate = 1e-3
        self.prate = 1e-3
        self.warmup = 100
        self.discount = 0.99
        self.bsize = 64
        self.rmsize = 10000
        self.window_length = 1
        self.tau = 0.001
        self.ou_theta = 0.15
        self.ou_sigma = 0.2
        self.ou_mu = 0.0
        self.validate_episodes = 20
        self.max_episode_length = 50
        self.validate_steps = 2000
        self.output = 'output'
        self.debug = True
        self.init_w = 0.003
        self.train_iter = 15000
        # self.epsilon = 50000
        self.epsilon = 10000
        self.seed = -1
        self.resume = 'default'
        self.k_ratio = 1e-6
        # we pass in the pre_generated action space
        self.action_space = None

args = Arguments()

In [25]:
args.output = get_output_folder(args.output, args.env)
if args.resume == 'default':
    args.resume = 'WordCountingDDPG/{}-run0'.format(args.env)

# env = NormalizedEnv(gym.make(args.env))
# env = gym.make(args.env)
env = WordCountingEnv()
args.low = env.action_space.low
args.high = env.action_space.high

if args.seed > 0:
    np.random.seed(args.seed)
    env.seed(args.seed)

nb_states = env.observation_space.shape[0]
nb_actions = env.action_space.shape[0]

agent = DDPG(nb_states, nb_actions, args)
evaluate = Evaluator(args.validate_episodes, 
    args.validate_steps, args.output, max_episode_length=args.max_episode_length)

start_time = time.time()

if args.mode == 'train':
    train(args.train_iter, agent, env, None, 
        args.validate_steps, args.output, max_episode_length=args.max_episode_length, debug=args.debug)
    end_time = time.time()

elif args.mode == 'test':
    test(args.validate_episodes, agent, env, evaluate, args.resume,
        visualize=True, debug=args.debug)

else:
    raise RuntimeError('undefined mode {}'.format(args.mode))

Building Topology
spout 2
WordCount 9
Database 9
torch.Size([128, 17])
torch.Size([128, 17])
 #0: episode_reward: -59.72908698666748 steps:50
 #1: episode_reward: -60.09435340167054 steps:100
 #2: episode_reward: -62.433107756642436 steps:150
 #3: episode_reward: -65.66412526994246 steps:200
 #4: episode_reward: -60.80776441155581 steps:250
 #5: episode_reward: -59.31427835493247 steps:300
 #6: episode_reward: -61.297119546701644 steps:350
 #7: episode_reward: -60.5946272117671 steps:400
 #8: episode_reward: -57.82810142501946 steps:450
 #9: episode_reward: -59.81715822192977 steps:500
 #10: episode_reward: -59.87107421169663 steps:550
 #11: episode_reward: -59.98502132176062 steps:600
 #12: episode_reward: -61.73739590311881 steps:650
 #13: episode_reward: -63.345780573191476 steps:700
 #14: episode_reward: -62.846388709987515 steps:750
 #15: episode_reward: -62.35977637156438 steps:800
 #16: episode_reward: -60.34033058491008 steps:850
 #17: episode_reward: -58.57586939998642 steps:9

KeyboardInterrupt: 